## Tabla de contenidos 

1. [Contexto del modelo](#id1)

    1.1. [Librerías utilizadas](#id2)

2. [Preparación del dataset](#id3)
    
3. [Construcción del modelo](#id4)

4. [Obtención de las etiquetas](#id5)

5. [Evaluación del modelo](#id6)
   
    5.1. [Conclusiones](#id7)

6. [Exportación del modelo](#id8)

<div id='id1' />

# 1. Contexto del modelo

El modelo en cuestión es un Kmeans y su objetivo es agrupar los comentarios en clusters similares en función de su contenido textual. Esto podría revelar patrones ocultos en los datos y agrupar comentarios que son semánticamente similares, lo cual podría ser útil para varias aplicaciones como la moderación de contenido o la comprensión de la estructura del corpus de comentarios.

<div id='id2' />

## 1.1. Librerías utilizadas

In [10]:
# Tratamiento de datos

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Gráficas

import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt

# Preprocesamiento y modelado

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

<div id='id3' />

# 2. Preparación del dataset

In [2]:
comments_df_undersampled = pd.read_csv('../01_data/02_processed/comments_df_undersampled.csv')
comments_df_undersampled

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,11ec6078ad3cd031,\n\nsee in my userpage the articles ive create...,0,0,0,0,0,0
1,11c0246ef55ef048,\n\nmatt fax page being deleted\n\nhi you just...,0,0,0,0,0,0
2,278ec4e59398a50b,this article contains information soursed from...,0,0,0,0,0,0
3,c2788f8bdaefeb13,24 promo \n\ni was really confused when i saw ...,0,0,0,0,0,0
4,fa04aa41f79c7884,i agree hes a biased editor 1141791837,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
32445,fef4cf7ba0012866,\n\n our previous conversation \n\nyou fucking...,1,0,1,0,1,1
32446,ff39a2895fc3b40e,you are a mischievious pubic hair,1,0,0,0,1,0
32447,ffa33d3122b599d6,your absurd edits \n\nyour absurd edits on gre...,1,0,1,0,1,0
32448,ffb47123b2d82762,\n\nhey listen dont you ever delete my edits e...,1,0,0,0,1,0


<div id='id4' />

# 3. Construcción del modelo

In [3]:

# Vectorizamos de los comentarios y creamos el set de datos

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

X = vectorizer.fit_transform(comments_df_undersampled['comment_text'])

In [4]:
# Normalización de los vectores

normalizer = Normalizer()
X_normalized = normalizer.fit_transform(X)

In [14]:
# Instanciación del modelo de clustering (K-Means)

num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

In [15]:
# Generamos el pipeline con las dos instancias anteriores y lo entrenamos

pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('normalizer', normalizer),
    ('kmeans', kmeans)
])

pipeline.fit(comments_df_undersampled['comment_text'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=5000, stop_words='english')),
                ('normalizer', Normalizer()),
                ('kmeans', KMeans(n_clusters=6, random_state=42))])

<div id='id5' />

# 4. Obtención de las etiquetas

In [16]:
# Obtenemos las etiquetas de cluster asignadas a cada comentario

cluster_labels = pipeline.named_steps['kmeans'].labels_

<div id='id6' />

# 5. Evaluación del modelo

In [17]:

# Análisis de resultados
comments_df_undersampled['cluster_label'] = cluster_labels

# Ver los comentarios agrupados por clusters
for cluster_id in range(num_clusters):
    cluster_comments = comments_df_undersampled[comments_df_undersampled['cluster_label'] == cluster_id]['comment_text'].tolist()
    print(f"Cluster {cluster_id}:")
    for comment in cluster_comments[:5]:  # Mostrar los primeros 5 comentarios de cada cluster
        print(comment)
    print("\n")

Cluster 0:
this article contains information soursed from the huddersfield aa street by street
this is a page for discussing improvements to the article about károly bund or as they say in hungary bund károly
im not sure im making my point clear so ill try a different explanation i feel that it is important that this page makes clear the difference between what the book actually says and what people hopefully reliable sourcessay it means interpretations should not be mixed in with the synopsis or character profiles in such a way that they cannot be told apart even what kesey himself said later about the book should be clearly differentiated from what the book itself actually says 
in regard to wether i think the lifeguard has brain damage the book certainly doesnt say so as it does in the case of pete bancini so why leave the reader of this article the impression that it does what would be wrong with just explaining that hes got cleatmarks on his forehead and letting the reader reach h

In [18]:
# Calcular métricas de evaluación

X_dense = X.toarray()

silhouette_avg = silhouette_score(X_dense, cluster_labels)
davies_bouldin = davies_bouldin_score(X_dense, cluster_labels)
calinski_harabasz = calinski_harabasz_score(X_dense, cluster_labels)

print(f"Silhouette Score: {silhouette_avg}")
print(f"Davies-Bouldin Score: {davies_bouldin}")
print(f"Calinski-Harabasz Score: {calinski_harabasz}")

Silhouette Score: 0.004651475886487441
Davies-Bouldin Score: 8.443094425860965
Calinski-Harabasz Score: 129.35730562494936


<div id='id7' />

## 5.1. Conclusión

Atendiendo a los valores de métricas de evaluación del clustering se pueden sacar las siguientes conclusiones:

1. **Silhouette Score**: Este valor indica cuán similares son las muestras dentro de cada cluster y cuán diferentes son entre clusters. Un Silhouette Score cercano a 0 indica una superposición de clusters, mientras que valores cercanos a 1 indican clusters bien definidos. En tu caso, un valor de 0.0064 sugiere una baja cohesión y separación entre los clusters, lo que podría indicar que las muestras dentro de los clusters son muy similares a las de otros clusters cercanos.

2. **Davies-Bouldin Score**: Esta métrica evalúa la compacidad dentro de los clusters y la separación entre los clusters. Un valor más bajo indica clusters mejor definidos y más separados entre sí. Con un valor de 8.44, se interpreta que los clusters están relativamente dispersos y no están bien definidos, lo cual puede ser problemático para interpretar los grupos formados.

3. **Calinski-Harabasz Score**: Este score mide la relación entre la dispersión dentro de los clusters y la dispersión entre los clusters. Un valor más alto indica clusters más densos y bien separados. Con un valor de 129.36, indica que existe una buena separación entre los clusters y que las muestras dentro de cada cluster son relativamente compactas en comparación con otras muestras.

En resumen, los valores de las métricas sugieren que los clusters no están bien definidos y hay una superposición significativa entre ellos (bajo Silhouette Score y alto Davies-Bouldin Score). Sin embargo, la alta puntuación en Calinski-Harabasz indica que existe alguna estructura en los clusters, pero se recomendaría investigar más para mejorar la calidad del clustering, quizás ajustando los parámetros del algoritmo de clustering o explorando técnicas de preprocesamiento de datos adicionales para obtener clusters más coherentes y distintos.

<div id='id8' />

# 6. Exportación del modelo

In [19]:
# Definimos el nombre del archivo
filename = '05_cuarto_modelo.pkl'

# Guardamos el pipeline en un archivo pickle

with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)